In [84]:
# https://stackoverflow.com/questions/59008756/getting-google-search-result-urls-from-search-string-or-url
import requests
from bs4 import BeautifulSoup

url = "http://www.google.com/search?q={}&hl=en"

def scrape_links(query):
    res = requests.get(url.format(query.replace(" ","+")),headers={"User-Agent":"Mozilla/5.0"})
    soup = BeautifulSoup(res.text,"lxml")
    for result in soup.findAll('a'):
        print(result.get('href'))

scrape_links('python')

/?sa=X&ved=0ahUKEwjW35C157LtAhWaHzQIHV8mA9kQOwgC
/?hl=en&output=search&sa=X&ved=0ahUKEwjW35C157LtAhWaHzQIHV8mA9kQPAgE
/search?q=python&hl=en&gbv=1&sei=k1_JX5bKLZq_0PEP38yMyA0
/search?q=python&hl=en&source=lnms&tbm=isch&sa=X&ved=0ahUKEwjW35C157LtAhWaHzQIHV8mA9kQ_AUICCgB
/search?q=python&hl=en&source=lnms&tbm=vid&sa=X&ved=0ahUKEwjW35C157LtAhWaHzQIHV8mA9kQ_AUICSgC
/search?q=python&hl=en&source=lnms&tbm=bks&sa=X&ved=0ahUKEwjW35C157LtAhWaHzQIHV8mA9kQ_AUICigD
http://maps.google.com/maps?q=python&hl=en&um=1&ie=UTF-8&sa=X&ved=0ahUKEwjW35C157LtAhWaHzQIHV8mA9kQ_AUICygE
/search?q=python&hl=en&source=lnms&tbm=nws&sa=X&ved=0ahUKEwjW35C157LtAhWaHzQIHV8mA9kQ_AUIDCgF
/search?q=python&hl=en&source=lnms&tbm=shop&sa=X&ved=0ahUKEwjW35C157LtAhWaHzQIHV8mA9kQ_AUIDSgG
/advanced_search
/search?q=python&hl=en&source=lnt&tbs=qdr:h&sa=X&ved=0ahUKEwjW35C157LtAhWaHzQIHV8mA9kQpwUIDw
/search?q=python&hl=en&source=lnt&tbs=qdr:d&sa=X&ved=0ahUKEwjW35C157LtAhWaHzQIHV8mA9kQpwUIEA
/search?q=python&hl=en&source=lnt&tbs=qdr: